In [8]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import log_loss, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector, ColumnTransformer

In [9]:
hr = pd.read_csv("D:/Training/Academy/ML(Python)/Cases/human-resources-analytics/HR_comma_sep.csv")
X, y = hr.drop('left', axis=1), hr['left']
ohe = OneHotEncoder(drop='first', sparse_output=False).set_output(transform='pandas')
col_trnf = ColumnTransformer([('OHE',ohe, make_column_selector(dtype_include=object) )],
                             remainder='passthrough',
                             verbose_feature_names_out=False)
col_trnf = col_trnf.set_output(transform='pandas')
X = col_trnf.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=25,
                                                   stratify=y)
scl_x = MinMaxScaler()
X_trn_scl, X_tst_scl = scl_x.fit_transform(X_train), scl_x.transform(X_test)

In [10]:
intializer = tf.keras.initializers.GlorotUniform(seed = 25)
tf.random.set_seed(25)
model = tf.keras.Sequential([tf.keras.Input(shape=(X_trn_scl.shape[1],)),
                             tf.keras.layers.Dense(14, activation='relu',
                                                   kernel_initializer=intializer),
                             tf.keras.layers.Dense(9, activation='relu',
                                                   kernel_initializer=intializer),
                             tf.keras.layers.Dense(4, activation='relu',
                                                   kernel_initializer=intializer),
                             tf.keras.layers.Dense(1, activation='sigmoid',
                                                   kernel_initializer=intializer)])
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
               loss=tf.keras.losses.BinaryCrossentropy())
history = model.fit(X_trn_scl, y_train,validation_data=(X_tst_scl,y_test), epochs=50)

Epoch 1/50
328/328 [==============================] - 2s 4ms/step - loss: 0.5951 - val_loss: 0.5197
Epoch 2/50
328/328 [==============================] - 1s 3ms/step - loss: 0.4864 - val_loss: 0.4487
Epoch 3/50
328/328 [==============================] - 1s 3ms/step - loss: 0.4281 - val_loss: 0.3978
Epoch 4/50
328/328 [==============================] - 1s 3ms/step - loss: 0.3781 - val_loss: 0.3461
Epoch 5/50
328/328 [==============================] - 1s 3ms/step - loss: 0.3325 - val_loss: 0.3058
Epoch 6/50
328/328 [==============================] - 1s 3ms/step - loss: 0.2999 - val_loss: 0.2808
Epoch 7/50
328/328 [==============================] - 1s 3ms/step - loss: 0.2782 - val_loss: 0.2658
Epoch 8/50
328/328 [==============================] - 1s 3ms/step - loss: 0.2649 - val_loss: 0.2535
Epoch 9/50
328/328 [==============================] - 1s 3ms/step - loss: 0.2525 - val_loss: 0.2481
Epoch 10/50
328/328 [==============================] - 1s 3ms/step - loss: 0.2428 - val_loss: 0.2572

In [11]:
y_pred_prob = model.predict(X_tst_scl)
print(log_loss(y_test, y_pred_prob))

141/141 [==============================] - 0s 2ms/step
0.1546937870476004


In [12]:
y_pred = np.where( y_pred_prob >= 0.5, 1, 0 )
print(accuracy_score(y_test, y_pred))

0.9537675038897533
